# PyGEM regional glacier change analysis

Brandon Tober, David Rounce<br>
Carnegie Mellon University<br>
20250103<br><br>
Objective: Showcase how to regionally compile simulations and analyze the aggregated glacier change.

>**Note:** in this notebook, any PyGEM Python scripts that are called will be preceeded by ```!```. In Jupyter Notebook, the ```!``` character is used to execute shell commands directly from a notebook cell. When you prefix a command with ```!```, Jupyter sends it to the system's command-line shell for execution, instead of interpreting it as Python code. Command-line arguments are also bracketed (e.g., ```{arg}```), when passing Python variables to a command-line shell from within a Jupyter Notebook. If executing a given PyGEM script directly from one's terminal, remove the ```!``` character and brackets ```{}``` around arguments.
>
>**Also note:** some useful information for any PyGEM script can be displayed by running
```!script -h```

This notebook follows the notebooks *run_calibration.ipynb* and *run_simulation.ipynb*, assuming that a set of regional PyGEM simulations have been performed for RGI region 6 (Iceland) for GCM BCC-CSM2-MR, scenario ssp245 following **MCMC** calibration.

In [ ]:
### imports ###
import os, sys, glob
# pygem imports
import pygem.setup.config as config
# check for config
config.ensure_config()
# read the config
pygem_prms = config.read_config()   # NOTE: ensure that your root path in ~/PyGEM/config.yaml points to
                                    # the appropriate location. If any errors occur, check this first.

## Compile regional simulations
The first step is to aggregate/compile all simulations for the desired RGI region.

In [ ]:
# take a look how to call the compile_simulations script:
!compile_simulations -h 

In [ ]:
# define args
gcm = 'BCC-CSM2-MR'
scenario = 'ssp245'
region = 6  # Iceland
calib = 'MCMC'  # `MCMC` calibration opt
startyr = 2000  # simulations start year
endyr = 2100    # simulations end year

In [ ]:
!compile_simulations -rgi_region01 {region} -gcm_name {gcm} -scenario {scenario} -gcm_startyear {startyr} -gcm_endyear {endyr} -option_calibration {calib}

The result of this step will be an aggregated netcdf file for variable, stored within `pygem_prms['root']/Output/simulations/compile`

>**Note:**, for monthly variables (e.g. `glac_massbaltotal_monthly`) simulations are aggregated in 1000-glacier chunks to keep file sizes manageable.
>
>The cell above will aggregate all available variables for the specified batch of simulations. The `-vars` argument of PyGEMs *compile_simulations.py* script shows the variables which can can be aggregated across simulations. However, some of these are not standard outputs and are only stored when *run_simulations.py* is passed the `-export_extra_vars` flag, or when `['sim']['out']['export_extra_vars']` is set to `true` in *~/PyGEM/config.yaml*. If the variables desired to be aggregated are not specified, *compile_simulations.py* will default to aggregating all available variables which were stored across simulations.
>
>If only specific variables are required, these could have been passed as arguments (space-delimited), e.g. for compiling glacier mass, area, and runoff:


```bash
!compile_simulations -rgi_region01 {region} -gcm_name {gcm} -scenario {scenario} -gcm_startyear {startyr} -gcm_endyear {endyr} -option_calibration {calib} -vars glac_mass_monthly glac_area_annual glac_runoff_monthly
```

## Regional analysis

Regional analysis can now be completed for the variables of interest.  Below we'll demonstate an analysis of regional glacier area, mass, and runoff.